# DateTime and DateTimeOffset Examples

Explore how both objects are used.

In [37]:
using System;
using System.Globalization;

// Helpers to print datetime
const int c1 = 30;
const int c2 = 13;
const int c3 = 20;

public void Print(DateTime dt)
{
    Console.WriteLine($"{dt,c1} {dt.Kind, c2} {dt.Ticks, c3}");
}

public void Print(DateTimeOffset dt)
{
    Console.WriteLine($"{dt,c1} {dt.Offset, c2} {dt.Ticks, c3}");
}

// This shows that MinValue and MaxValue are the same for both objects
Print(DateTime.MinValue);
Print(DateTime.MaxValue);
Print(DateTimeOffset.MinValue);
Print(DateTimeOffset.MaxValue);


          1/1/0001 12:00:00 AM   Unspecified                    0
        12/31/9999 11:59:59 PM   Unspecified  3155378975999999999
   1/1/0001 12:00:00 AM +00:00      00:00:00                    0
 12/31/9999 11:59:59 PM +00:00      00:00:00  3155378975999999999


Let's understand conversion between these two objects

In [38]:
// This shows when converted, the number of ticks are still the same
var utcNow = DateTime.UtcNow;
Print(utcNow);
Print(new DateTimeOffset(utcNow));

          8/13/2023 1:35:56 PM           Utc   638275305566318348
   8/13/2023 1:35:56 PM +00:00      00:00:00   638275305566318348


In [39]:
// This shows we can only convert to DateTimeOffset with the same UTC offset 
// this example here will throw exception
utcNow = DateTime.UtcNow;
var offsetNow = new DateTimeOffset(utcNow, TimeSpan.FromHours(10));
Print(utcNow);
Print(offsetNow);

Error: System.ArgumentException: The UTC Offset for Utc DateTime instances must be 0. (Parameter 'offset')
   at System.DateTimeOffset..ctor(DateTime dateTime, TimeSpan offset)
   at Submission#40.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [40]:
// No exceptions when we use an offset of 0
utcNow = DateTime.UtcNow;
var offsetNow = new DateTimeOffset(utcNow, TimeSpan.FromHours(0));
Print(utcNow);
Print(offsetNow);

          8/13/2023 1:35:56 PM           Utc   638275305568532521
   8/13/2023 1:35:56 PM +00:00      00:00:00   638275305568532521


It is important to take note of DateTimeKind when creating a new DateTime (new/from/parse)

Conversion to DateTimeOffset can trigger fundamental changes when DateTimeKind is not Unspecified

In [51]:
var datetimeStr = "15/08/2020 2:00:00 PM";

var dt1 = new DateTime(2020, 08, 15, 14, 00, 00);
var dt2 = DateTime.ParseExact(datetimeStr, "dd/MM/yyyy h:mm:ss tt", CultureInfo.InvariantCulture);
var dt3 = DateTime.Now;
Print(dt1);
Print(dt2);
Print(dt3); //only this is local

// converting to another offset
Print(new DateTimeOffset(dt1, TimeSpan.FromHours(13))); // no. of ticks don't change
Print(new DateTimeOffset(dt2, TimeSpan.FromHours(13)));
Print(new DateTimeOffset(dt1, TimeSpan.FromHours(03))); // no. of ticks don't change
Print(new DateTimeOffset(dt2, TimeSpan.FromHours(03)));
Print(new DateTimeOffset(dt1, TimeSpan.FromHours(09))); // no. of ticks don't change
Print(new DateTimeOffset(dt2, TimeSpan.FromHours(09)));
Print(new DateTimeOffset(dt3, DateTimeOffset.Now.Offset)); 
// when DateTime.Unspecified, we can freely change to any offset without ticks changing
// the datetime component is preserved perfectly


          8/15/2020 2:00:00 PM   Unspecified   637330968000000000
          8/15/2020 2:00:00 PM   Unspecified   637330968000000000
          8/13/2023 9:41:32 PM         Local   638275596928386917
   8/15/2020 2:00:00 PM +13:00      13:00:00   637330968000000000
   8/15/2020 2:00:00 PM +13:00      13:00:00   637330968000000000
   8/15/2020 2:00:00 PM +03:00      03:00:00   637330968000000000
   8/15/2020 2:00:00 PM +03:00      03:00:00   637330968000000000
   8/15/2020 2:00:00 PM +09:00      09:00:00   637330968000000000
   8/15/2020 2:00:00 PM +09:00      09:00:00   637330968000000000
   8/13/2023 9:41:32 PM +08:00      08:00:00   638275596928386917


DateTime's DateTimeKind conversion

In [55]:
dt1 = DateTime.Now;
// local to utc will preserve ticks
Print(dt1);
Print(dt1.ToUniversalTime());

// from unspecified to local/utc will not preserve ticks
dt1 = DateTime.SpecifyKind(DateTime.Now, DateTimeKind.Unspecified);
Print(dt1);
Print(dt1.ToLocalTime());
Print(dt1.ToUniversalTime());

// will not end up with same result as above, this shows the importance of DateTimeKind
dt1 = DateTime.SpecifyKind(DateTime.UtcNow, DateTimeKind.Unspecified);
Print(dt1);
Print(dt1.ToLocalTime());
Print(dt1.ToUniversalTime());


         8/13/2023 10:02:38 PM         Local   638275609588139240
          8/13/2023 2:02:38 PM           Utc   638275321588139240
         8/13/2023 10:02:38 PM   Unspecified   638275609588160802
          8/14/2023 6:02:38 AM         Local   638275897588160802
          8/13/2023 2:02:38 PM           Utc   638275321588160802
          8/13/2023 2:02:38 PM   Unspecified   638275321588166202
         8/13/2023 10:02:38 PM         Local   638275609588166202
          8/13/2023 6:02:38 AM           Utc   638275033588166202


Converting DateTime.Now to DateTimeOffset will never be 100% safe.

Because DateTime.Now defaults to DateTimeKind.Local and if the OS/Container does not adjust to correct timezone then you will end up with issues.

i.e. var offset = new DateTimeOffset(DateTime.Now, TimeSpan.FromHours(8)); --> this will not work if OS/Container defaulted to UTC timezone.

you can try new DateTimeOffset(DateTime.Now, DateTimeOffset.Now.Offset); --> won't throw but you will still end up with wrong offset.

This is also a problem when you are running dotnet test in pipelines, the running container may not be configured to your timezone offset.

In [57]:
// Consider the source of your datetime.
// in this instance we assume it's the system clock

// first way; this may look dumb but it's safe if somehow you need to start with DateTime so start with UtcNow
var ddt1 = DateTime.UtcNow;
var ddto1 = new DateTimeOffset(ddt1, TimeSpan.FromHours(0));
var final = ddto1.ToOffset(TimeSpan.FromHours(8));
Print(final);

// second way; directly use DateTimeOffset would be more straightforward
ddto1 = DateTimeOffset.UtcNow;
final = ddto1.ToOffset(TimeSpan.FromHours(8));
Print(final);



  8/13/2023 10:13:45 PM +08:00      08:00:00   638275616255802312
  8/13/2023 10:13:45 PM +08:00      08:00:00   638275616255805871


Any time you need a timezone offset, just go straight with DateTimeOffset.

Any use of DateTime would most likely cause issues downstream.

Always test your code with different system clock, this will help you understand if code is vulnerable to system clock changes.